# Systematic Macro Credit — End-to-End Workflow

This notebook demonstrates the complete workflow for the CDX overlay strategy:

1. **Data Generation**: Create synthetic CDX, VIX, and ETF data
2. **Data Loading**: Load and validate market data
3. **Signal Computation**: Calculate individual signals and composite score
4. **Backtesting**: Run strategy backtest with position tracking
5. **Visualization**: Analyze results with interactive plots

All components use deterministic random seeds for reproducibility.

## 1. Setup and Imports

In [1]:
import logging
from pathlib import Path

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Import project modules
from macrocredit.data.sample_data import (
    generate_cdx_sample,
    generate_vix_sample,
    generate_etf_sample,
)
from macrocredit.models.signals import (
    compute_cdx_etf_basis,
    compute_cdx_vix_gap,
    compute_spread_momentum,
)
from macrocredit.models.aggregator import aggregate_signals
from macrocredit.models.config import SignalConfig
from macrocredit.backtest.config import BacktestConfig
from macrocredit.backtest.engine import run_backtest
from macrocredit.backtest.metrics import compute_performance_metrics
from macrocredit.visualization.plots import (
    plot_signal,
    plot_equity_curve,
    plot_drawdown,
    plot_exposures,
)

print("✅ Imports successful")

✅ Imports successful


## 2. Generate Sample Data

Create synthetic market data with realistic dynamics:
- **CDX IG 5Y**: Investment grade credit spreads
- **VIX**: Equity volatility index
- **HYG ETF**: High yield credit ETF

In [2]:
# Configuration
START_DATE = "2023-01-01"
PERIODS = 252  # One year of trading days
SEED = 42

# Generate CDX data
cdx_data = generate_cdx_sample(
    start_date=START_DATE,
    periods=PERIODS,
    index_name="CDX_IG",
    tenor="5Y",
    base_spread=70.0,
    volatility=3.0,
    seed=SEED,
)

# Generate VIX data
vix_data = generate_vix_sample(
    start_date=START_DATE,
    periods=PERIODS,
    base_vix=16.0,
    volatility=2.0,
    seed=SEED + 1,
)

# Generate ETF data
etf_data = generate_etf_sample(
    start_date=START_DATE,
    periods=PERIODS,
    ticker="HYG",
    base_price=75.0,
    volatility=0.6,
    seed=SEED + 2,
)

print(f"✅ Generated {len(cdx_data)} days of CDX data")
print(f"✅ Generated {len(vix_data)} days of VIX data")
print(f"✅ Generated {len(etf_data)} days of ETF data")

# Display sample data
print("\nCDX Sample:")
print(cdx_data.head())
print("\nVIX Sample:")
print(vix_data.head())
print("\nETF Sample:")
print(etf_data.head())

2025-10-28 22:56:25,703 - macrocredit.data.sample_data - INFO - Generating CDX sample: index=CDX_IG, tenor=5Y, periods=252
2025-10-28 22:56:25,707 - macrocredit.data.sample_data - INFO - Generating VIX sample: periods=252
2025-10-28 22:56:25,710 - macrocredit.data.sample_data - INFO - Generating ETF sample: ticker=HYG, periods=252
2025-10-28 22:56:25,707 - macrocredit.data.sample_data - INFO - Generating VIX sample: periods=252
2025-10-28 22:56:25,710 - macrocredit.data.sample_data - INFO - Generating ETF sample: ticker=HYG, periods=252


✅ Generated 252 days of CDX data
✅ Generated 252 days of VIX data
✅ Generated 252 days of ETF data

CDX Sample:
        date     spread      index tenor  series
0 2023-01-01  70.000000  CDX_IG_5Y    5Y      42
1 2023-01-02  70.914151  CDX_IG_5Y    5Y      42
2 2023-01-03  67.702784  CDX_IG_5Y    5Y      42
3 2023-01-04  70.183859  CDX_IG_5Y    5Y      42
4 2023-01-05  72.987167  CDX_IG_5Y    5Y      42

VIX Sample:
        date      close       open       high        low
0 2023-01-01  16.000000  16.000000  16.711285  15.583029
1 2023-01-02  17.356357  16.000000  17.501090  17.341342
2 2023-01-03  26.561353  17.356357  26.708790  25.871609
3 2023-01-04  25.010464  26.561353  25.104670  23.436554
4 2023-01-05  22.091705  25.010464  22.188644  21.507409

ETF Sample:
        date      close ticker   volume       open       high        low
0 2023-01-01  75.880203    HYG  6007541  75.880203  76.129997  75.671493
1 2023-01-02  75.949926    HYG  4304295  75.880203  75.958874  75.867396
2 2023-

## 3. Merge Market Data

Use standard pandas operations to combine data sources.

In [3]:
# Prepare data for merging
cdx_prep = cdx_data[["date", "spread"]].rename(columns={"spread": "cdx_spread"})
vix_prep = vix_data[["date", "close"]].rename(columns={"close": "vix_close"})
etf_prep = etf_data[["date", "close"]].rename(columns={"close": "etf_close"})

# Merge on date
market_data = cdx_prep.merge(vix_prep, on="date").merge(etf_prep, on="date")
market_data = market_data.set_index("date")

print(f"✅ Merged market data: {len(market_data)} rows")
print(f"Columns: {list(market_data.columns)}")
print("\nMerged Data Sample:")
print(market_data.head())

✅ Merged market data: 252 rows
Columns: ['cdx_spread', 'vix_close', 'etf_close']

Merged Data Sample:
            cdx_spread  vix_close  etf_close
date                                        
2023-01-01   70.000000  16.000000  75.880203
2023-01-02   70.914151  17.356357  75.949926
2023-01-03   67.702784  26.561353  76.156336
2023-01-04   70.183859  25.010464  76.859569
2023-01-05   72.987167  22.091705  77.375789


## 4. Compute Individual Signals

Calculate three cross-asset signals:

1. **CDX-ETF Basis**: Spread between CDX and implied credit from ETF prices
2. **CDX-VIX Gap**: Divergence between equity vol and credit spreads
3. **Spread Momentum**: Short-term trend in CDX spreads

**Signal Convention**: Positive = Long credit risk (buy CDX/sell protection)

In [4]:
# Prepare DataFrames for signal computation
cdx_df = market_data[["cdx_spread"]].rename(columns={"cdx_spread": "spread"})
vix_df = market_data[["vix_close"]].rename(columns={"vix_close": "close"})
etf_df = market_data[["etf_close"]].rename(columns={"etf_close": "close"})

# Compute CDX-ETF basis signal
cdx_etf_basis = compute_cdx_etf_basis(
    cdx_df=cdx_df,
    etf_df=etf_df,
)

# Compute CDX-VIX gap signal
cdx_vix_gap = compute_cdx_vix_gap(
    cdx_df=cdx_df,
    vix_df=vix_df,
)

# Compute spread momentum signal
spread_momentum = compute_spread_momentum(
    cdx_df=cdx_df,
)

# Add signals to market data
market_data["cdx_etf_basis"] = cdx_etf_basis
market_data["cdx_vix_gap"] = cdx_vix_gap
market_data["spread_momentum"] = spread_momentum

print("✅ Computed individual signals")
print("\nSignal Statistics:")
print(market_data[["cdx_etf_basis", "cdx_vix_gap", "spread_momentum"]].describe())


2025-10-28 22:56:25,739 - macrocredit.models.signals - INFO - Computing CDX-ETF basis: cdx_rows=252, etf_rows=252, lookback=20
2025-10-28 22:56:25,741 - macrocredit.models.signals - INFO - Computing CDX-VIX gap: cdx_rows=252, vix_rows=252, lookback=20
2025-10-28 22:56:25,742 - macrocredit.models.signals - INFO - Computing spread momentum: cdx_rows=252, lookback=20
2025-10-28 22:56:25,741 - macrocredit.models.signals - INFO - Computing CDX-VIX gap: cdx_rows=252, vix_rows=252, lookback=20
2025-10-28 22:56:25,742 - macrocredit.models.signals - INFO - Computing spread momentum: cdx_rows=252, lookback=20


✅ Computed individual signals

Signal Statistics:
       cdx_etf_basis  cdx_vix_gap  spread_momentum
count     243.000000   234.000000       232.000000
mean       -0.101078    -0.013305         0.010373
std         1.167607     1.118674         1.932439
min        -2.772213    -2.886740        -5.711461
25%        -0.956350    -0.831705        -1.408021
50%        -0.078569     0.108475        -0.040037
75%         0.666807     0.779734         1.424015
max         3.328297     3.132649         5.096826


## 5. Aggregate Signals into Composite Score

Combine individual signals using weighted aggregation.

In [5]:
from macrocredit.models.config import AggregatorConfig

# Configure signal weights
aggregator_config = AggregatorConfig(
    cdx_etf_basis_weight=0.4,
    cdx_vix_gap_weight=0.3,
    spread_momentum_weight=0.3,
)

# Aggregate signals
composite_signal = aggregate_signals(
    cdx_etf_basis=market_data["cdx_etf_basis"],
    cdx_vix_gap=market_data["cdx_vix_gap"],
    spread_momentum=market_data["spread_momentum"],
    config=aggregator_config,
)

market_data["composite_signal"] = composite_signal

print("✅ Aggregated composite signal")
print(f"\nSignal Weights: {aggregator_config}")
print("\nComposite Signal Statistics:")
print(market_data["composite_signal"].describe())


2025-10-28 22:56:25,758 - macrocredit.models.aggregator - INFO - Aggregating signals: basis_weight=0.40, vix_weight=0.30, mom_weight=0.30
2025-10-28 22:56:25,760 - macrocredit.models.aggregator - INFO - Composite signal: valid_obs=232, mean=-0.051, std=0.542
2025-10-28 22:56:25,760 - macrocredit.models.aggregator - INFO - Composite signal: valid_obs=232, mean=-0.051, std=0.542


✅ Aggregated composite signal

Signal Weights: AggregatorConfig(cdx_etf_basis_weight=0.4, cdx_vix_gap_weight=0.3, spread_momentum_weight=0.3, threshold=1.0)

Composite Signal Statistics:
count    232.000000
mean      -0.050797
std        0.541915
min       -1.577443
25%       -0.386113
50%       -0.109750
75%        0.330972
max        1.448275
Name: composite_signal, dtype: float64


## 6. Visualize Signals

Plot individual signals and composite score to understand strategy drivers.

In [6]:
# Plot composite signal
fig_signal = plot_signal(
    signal=market_data["composite_signal"],
    title="Composite Signal"
)
fig_signal.show()

print("✅ Signal visualization complete")

2025-10-28 22:56:25,770 - macrocredit.visualization.plots - INFO - Plotting signal: 252 observations


✅ Signal visualization complete


## 7. Run Backtest

Execute strategy backtest with position limits and transaction costs.

In [7]:
# Configure backtest
backtest_config = BacktestConfig(
    entry_threshold=1.5,
    exit_threshold=0.75,
    position_size=10.0,  # $10M notional
    transaction_cost_bps=1.0,  # 1 bps
)

# Run backtest
backtest_result = run_backtest(
    composite_signal=market_data["composite_signal"],
    spread=market_data["cdx_spread"],
    config=backtest_config,
)

print("✅ Backtest complete")
print(f"\nBacktest Results: {len(backtest_result.positions)} periods")
print("\nPosition Data Sample:")
print(backtest_result.positions.head())
print("\nP&L Data Sample:")
print(backtest_result.pnl.head())
print(f"\nFinal Cumulative P&L: ${backtest_result.pnl['cumulative_pnl'].iloc[-1]:,.2f}")


2025-10-28 22:56:26,101 - macrocredit.backtest.engine - INFO - Starting backtest: dates=252, entry_threshold=1.50, position_size=10.0MM


2025-10-28 22:56:26,111 - macrocredit.backtest.engine - INFO - Backtest complete: trades=1, total_pnl=$383426, avg_per_trade=$383426


✅ Backtest complete

Backtest Results: 232 periods

Position Data Sample:
              signal  position  days_held     spread
date                                                
2023-01-21  0.488390         0          0  71.129851
2023-01-22  0.687309         0          0  70.462279
2023-01-23  0.058557         0          0  68.373262
2023-01-24  0.519808         0          0  72.203560
2023-01-25  0.921330         0          0  71.519616

P&L Data Sample:
            spread_pnl  cost  net_pnl  cumulative_pnl
date                                                 
2023-01-21         0.0   0.0      0.0             0.0
2023-01-22         0.0   0.0      0.0             0.0
2023-01-23         0.0   0.0      0.0             0.0
2023-01-24         0.0   0.0      0.0             0.0
2023-01-25         0.0   0.0      0.0             0.0

Final Cumulative P&L: $383,425.54


## 8. Calculate Performance Metrics

Compute Sharpe ratio, max drawdown, and other key statistics.

In [8]:
# Calculate performance metrics
metrics = compute_performance_metrics(
    pnl_df=backtest_result.pnl,
    positions_df=backtest_result.positions,
)

print("✅ Performance metrics calculated")
print("\n" + "="*50)
print("PERFORMANCE SUMMARY")
print("="*50)
print(f"Total Return:        ${metrics.total_return:>12,.2f}")
print(f"Sharpe Ratio:        {metrics.sharpe_ratio:>12.2f}")
print(f"Sortino Ratio:       {metrics.sortino_ratio:>12.2f}")
print(f"Max Drawdown:        ${metrics.max_drawdown:>12,.2f}")
print(f"Calmar Ratio:        {metrics.calmar_ratio:>12.2f}")
print(f"Hit Rate:            {metrics.hit_rate:>12.1%}")
print(f"Number of Trades:    {metrics.n_trades:>12d}")
print(f"Avg Holding Days:    {metrics.avg_holding_days:>12.1f}")
print(f"Annualized Return:   ${metrics.annualized_return:>12,.2f}")
print(f"Annual Volatility:   ${metrics.annualized_volatility:>12,.2f}")
print("="*50)


2025-10-28 22:56:26,122 - macrocredit.backtest.metrics - INFO - Computing performance metrics
2025-10-28 22:56:26,126 - macrocredit.backtest.metrics - INFO - Metrics computed: sharpe=1.22, max_dd=$-1000, hit_rate=100.0%
2025-10-28 22:56:26,126 - macrocredit.backtest.metrics - INFO - Metrics computed: sharpe=1.22, max_dd=$-1000, hit_rate=100.0%


✅ Performance metrics calculated

PERFORMANCE SUMMARY
Total Return:        $  383,425.54
Sharpe Ratio:                1.22
Sortino Ratio:               0.00
Max Drawdown:        $   -1,000.00
Calmar Ratio:              416.48
Hit Rate:                  100.0%
Number of Trades:               1
Avg Holding Days:             0.5
Annualized Return:   $  416,479.47
Annual Volatility:   $  340,441.81


## 9. Visualize Backtest Results

Create interactive visualizations of positions, P&L, and performance.

In [9]:
import plotly.express as px

# Plot positions over time (simple line chart)
fig_positions = px.line(
    backtest_result.positions,
    y="position",
    title="Position History",
    labels={"position": "Position (+1=Long, -1=Short, 0=Flat)", "date": "Date"},
)
fig_positions.show()

print("✅ Position visualization complete")


✅ Position visualization complete


In [10]:
# Plot P&L evolution (equity curve)
fig_pnl = plot_equity_curve(
    pnl=backtest_result.pnl["net_pnl"],
    title="Cumulative P&L",
)
fig_pnl.show()

print("✅ P&L visualization complete")


2025-10-28 22:56:26,174 - macrocredit.visualization.plots - INFO - Plotting equity curve: 232 observations


✅ P&L visualization complete


In [11]:
# Plot drawdown analysis
fig_drawdown = plot_drawdown(
    pnl=backtest_result.pnl["net_pnl"],
    title="Drawdown Analysis",
)
fig_drawdown.show()

print("✅ Drawdown visualization complete")


2025-10-28 22:56:26,221 - macrocredit.visualization.plots - INFO - Plotting drawdown: 232 observations


✅ Drawdown visualization complete


## 10. Export Results

Save backtest results and metrics for further analysis.

In [12]:
from macrocredit.persistence.parquet_io import save_parquet
from macrocredit.persistence.json_io import save_json
from dataclasses import asdict

# Create output directory
output_dir = Path("data/processed")
output_dir.mkdir(parents=True, exist_ok=True)

# Save backtest results (positions and P&L)
positions_path = output_dir / "backtest_positions.parquet"
save_parquet(backtest_result.positions, positions_path)

pnl_path = output_dir / "backtest_pnl.parquet"
save_parquet(backtest_result.pnl, pnl_path)

# Save performance metrics
metrics_path = output_dir / "performance_metrics.json"
save_json(asdict(metrics), metrics_path)

print(f"✅ Results saved to {output_dir}")
print(f"   - Positions: {positions_path}")
print(f"   - P&L: {pnl_path}")
print(f"   - Metrics: {metrics_path}")


2025-10-28 22:56:26,271 - macrocredit.persistence.parquet_io - INFO - Saving DataFrame to Parquet: path=data\processed\backtest_positions.parquet, rows=232, columns=4, compression=snappy
2025-10-28 22:56:26,286 - macrocredit.persistence.parquet_io - INFO - Saving DataFrame to Parquet: path=data\processed\backtest_pnl.parquet, rows=232, columns=4, compression=snappy
2025-10-28 22:56:26,290 - macrocredit.persistence.json_io - INFO - Saving JSON to data\processed\performance_metrics.json (13 top-level keys)
2025-10-28 22:56:26,286 - macrocredit.persistence.parquet_io - INFO - Saving DataFrame to Parquet: path=data\processed\backtest_pnl.parquet, rows=232, columns=4, compression=snappy
2025-10-28 22:56:26,290 - macrocredit.persistence.json_io - INFO - Saving JSON to data\processed\performance_metrics.json (13 top-level keys)


✅ Results saved to data\processed
   - Positions: data\processed\backtest_positions.parquet
   - P&L: data\processed\backtest_pnl.parquet
   - Metrics: data\processed\performance_metrics.json


## Summary

This notebook demonstrated the complete end-to-end workflow:

1. ✅ Generated synthetic market data (CDX, VIX, ETF)
2. ✅ Computed cross-asset signals (basis, gap, momentum)
3. ✅ Aggregated signals into composite score
4. ✅ Executed systematic backtest with risk controls
5. ✅ Calculated performance metrics
6. ✅ Visualized results with interactive plots
7. ✅ Exported results for persistence

### Next Steps

- Experiment with different signal weights in `SignalConfig`
- Adjust position sizing and risk limits in `BacktestConfig`
- Test different CDX indices (HY, XO) and tenors
- Extend to multi-tenor strategies
- Add regime-dependent signal adjustments

### Key Takeaways

- All components are **modular** and **composable**
- Data generation is **deterministic** (reproducible with seeds)
- Signals follow **consistent sign convention** (positive = long credit)
- Visualization provides **actionable insights** into strategy behavior
- Framework supports **rapid iteration** on investment ideas